In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve

In [2]:
transactions = pd.read_csv("/content/drive/MyDrive/colab_data/kaggle_H&M/data/transactions_raw.csv")

ratings = transactions

In [16]:
ratings = transactions[-5000000:]
ratings

,t_dat,customer_id,article_id,price,sales_channel_id
26788324,2020-06-04,33107add0f51e74f63836f005ad841955de5655ef9b755...,863937003,0.011847,1
26788325,2020-06-04,33107add0f51e74f63836f005ad841955de5655ef9b755...,837430001,0.050831,1
26788326,2020-06-04,3322e39c235a1891d20c9f1c696f02c209cf5a5b2efbe7...,878013001,0.050831,2
26788327,2020-06-04,3322e39c235a1891d20c9f1c696f02c209cf5a5b2efbe7...,878013001,0.050831,2
26788328,2020-06-04,3325c5eefa1d842c572f8c66249e26e016c1fdca19c1da...,706016001,0.033881,1
...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1


In [17]:
ratings['rank_latest'] = ratings.groupby(['customer_id'])['t_dat'].rank(method = 'first', ascending = False)
# ratings = ratings.query(' rank_latest < 30')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
ratings

,t_dat,customer_id,article_id,price,sales_channel_id,rank_latest
26788324,2020-06-04,33107add0f51e74f63836f005ad841955de5655ef9b755...,863937003,0.011847,1,26.0
26788325,2020-06-04,33107add0f51e74f63836f005ad841955de5655ef9b755...,837430001,0.050831,1,27.0
26788326,2020-06-04,3322e39c235a1891d20c9f1c696f02c209cf5a5b2efbe7...,878013001,0.050831,2,8.0
26788327,2020-06-04,3322e39c235a1891d20c9f1c696f02c209cf5a5b2efbe7...,878013001,0.050831,2,9.0
26788328,2020-06-04,3325c5eefa1d842c572f8c66249e26e016c1fdca19c1da...,706016001,0.033881,1,1.0
...,...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2,1.0
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2,2.0
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1,1.0
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1,1.0


In [19]:
def map_column(df: pd.DataFrame, col_name: str):
    """
    Maps column values to integers
    :param df:
    :param col_name:
    :return:
    """
    values = sorted(list(df[col_name].unique()))
    mapping = {k: i for i, k in enumerate(values)}
    inverse_mapping = {v: k for k, v in mapping.items()}

    df[col_name + "_mapped"] = df[col_name].map(mapping)

    return df, mapping, inverse_mapping

ratings, cus_mapping, cus_inverse_mapping = map_column(ratings, col_name="customer_id")
ratings, art_mapping, art_inverse_mapping = map_column(ratings, col_name="article_id")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [20]:
ratings['purchase'] = 1

ratings

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,t_dat,customer_id,article_id,price,sales_channel_id,rank_latest,customer_id_mapped,article_id_mapped,purchase
26788324,2020-06-04,33107add0f51e74f63836f005ad841955de5655ef9b755...,863937003,0.011847,1,26.0,117687,35775,1
26788325,2020-06-04,33107add0f51e74f63836f005ad841955de5655ef9b755...,837430001,0.050831,1,27.0,117687,31000,1
26788326,2020-06-04,3322e39c235a1891d20c9f1c696f02c209cf5a5b2efbe7...,878013001,0.050831,2,8.0,117864,38969,1
26788327,2020-06-04,3322e39c235a1891d20c9f1c696f02c209cf5a5b2efbe7...,878013001,0.050831,2,9.0,117864,38969,1
26788328,2020-06-04,3325c5eefa1d842c572f8c66249e26e016c1fdca19c1da...,706016001,0.033881,1,1.0,117887,11586,1
...,...,...,...,...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2,1.0,590785,45149,1
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2,2.0,590785,41249,1
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1,1.0,590798,44356,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1,1.0,590812,30385,1


In [21]:

val_start_date = '2020-09-16'
train_ratings = ratings.query(f"t_dat < '{val_start_date}'").reset_index(drop=True)
valid_ratings = ratings.query(f"t_dat >= '{val_start_date}'").reset_index(drop=True)

In [22]:
train_ratings

,t_dat,customer_id,article_id,price,sales_channel_id,rank_latest,customer_id_mapped,article_id_mapped,purchase
0,2020-06-04,33107add0f51e74f63836f005ad841955de5655ef9b755...,863937003,0.011847,1,26.0,117687,35775,1
1,2020-06-04,33107add0f51e74f63836f005ad841955de5655ef9b755...,837430001,0.050831,1,27.0,117687,31000,1
2,2020-06-04,3322e39c235a1891d20c9f1c696f02c209cf5a5b2efbe7...,878013001,0.050831,2,8.0,117864,38969,1
3,2020-06-04,3322e39c235a1891d20c9f1c696f02c209cf5a5b2efbe7...,878013001,0.050831,2,9.0,117864,38969,1
4,2020-06-04,3325c5eefa1d842c572f8c66249e26e016c1fdca19c1da...,706016001,0.033881,1,1.0,117887,11586,1
...,...,...,...,...,...,...,...,...,...
4759684,2020-09-15,ffe41634ff990908faacbb465063e027e7c39499f8dfc1...,850917001,0.025407,1,3.0,590670,33026,1
4759685,2020-09-15,ffef8aec5cf011fa1393b40337a5993ce0b7b81af6b322...,853316001,0.008458,1,4.0,590761,33519,1
4759686,2020-09-15,ffef8aec5cf011fa1393b40337a5993ce0b7b81af6b322...,296366006,0.000847,1,5.0,590761,311,1
4759687,2020-09-15,ffef8aec5cf011fa1393b40337a5993ce0b7b81af6b322...,789769001,0.013542,1,6.0,590761,22542,1


In [23]:

# ratings['customer_id'] = ratings['customer_id'].astype(int)
# ratings = ratings[['customer_id','article_id','purchase']]
grouped_cleaned = ratings.groupby(['customer_id','article_id']).sum().reset_index()
grouped_cleaned

,customer_id,article_id,price,sales_channel_id,rank_latest,customer_id_mapped,article_id_mapped,purchase
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601043,0.050831,1,1.0,0,3502,1
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,826211002,0.027102,1,1.0,1,29286,1
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,794321007,0.061000,2,1.0,2,23129,1
3,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,730683050,0.033881,2,2.0,3,14197,1
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,791587015,0.020322,2,4.0,3,22797,1
...,...,...,...,...,...,...,...,...
4405305,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,795975009,0.011847,1,7.0,590904,23359,1
4405306,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,816759002,0.016932,2,8.0,590904,27471,1
4405307,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,835561003,0.010153,1,6.0,590904,30727,1
4405308,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,884081001,0.012898,2,3.0,590904,40061,1


In [24]:
ratings

,t_dat,customer_id,article_id,price,sales_channel_id,rank_latest,customer_id_mapped,article_id_mapped,purchase
26788324,2020-06-04,33107add0f51e74f63836f005ad841955de5655ef9b755...,863937003,0.011847,1,26.0,117687,35775,1
26788325,2020-06-04,33107add0f51e74f63836f005ad841955de5655ef9b755...,837430001,0.050831,1,27.0,117687,31000,1
26788326,2020-06-04,3322e39c235a1891d20c9f1c696f02c209cf5a5b2efbe7...,878013001,0.050831,2,8.0,117864,38969,1
26788327,2020-06-04,3322e39c235a1891d20c9f1c696f02c209cf5a5b2efbe7...,878013001,0.050831,2,9.0,117864,38969,1
26788328,2020-06-04,3325c5eefa1d842c572f8c66249e26e016c1fdca19c1da...,706016001,0.033881,1,1.0,117887,11586,1
...,...,...,...,...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2,1.0,590785,45149,1
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2,2.0,590785,41249,1
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1,1.0,590798,44356,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1,1.0,590812,30385,1


In [25]:
customers = list(np.sort(ratings['customer_id'].unique()))
articles = list (ratings['article_id'].unique())

## for training
purchase = list(ratings['purchase'])

rows = ratings['customer_id_mapped']
cols = ratings['article_id_mapped']

print(len(customers)) # 4327
print(len(articles))  # 3650

590906
45600


In [26]:
cols

26788324    35775
26788325    31000
26788326    38969
26788327    38969
26788328    11586
            ...  
31788319    45149
31788320    41249
31788321    44356
31788322    30385
31788323    42230
Name: article_id_mapped, Length: 5000000, dtype: int64

In [27]:
# csr: Compressed Sparse matrix by Row
purchase_sparse = sparse.csr_matrix((purchase, (rows, cols)), shape = (len(customers), len(articles)))
purchase_sparse #4327 * 3650 행렬

<590906x45600 sparse matrix of type '<class 'numpy.longlong'>'
	with 4405310 stored elements in Compressed Sparse Row format>

In [28]:
# Sparsity: 얼마나 비어있나?
matrix_size = purchase_sparse.shape[0]* purchase_sparse.shape[1]
num_purchases = len(purchase_sparse.nonzero()[0])
sparsity = 100 * (1 - (num_purchases / matrix_size))
sparsity

99.98365092325369

In [29]:
# import random
# def make_train (matrix, percentage = .2):
#     '''
#     -----------------------------------------------------
#     설명
#     유저-아이템 행렬 (matrix)에서 
#     1. 0이상의 값을 가지면 1의 값을 갖도록 binary하게 테스트 데이터를 만들고
#     2. 훈련 데이터는 원본 행렬에서 percentage 비율만큼 0으로 바뀜
    
#     -----------------------------------------------------
#     반환
#     training_set: 훈련 데이터에서 percentage 비율만큼 0으로 바뀐 행렬
#     test_set:     원본 유저-아이템 행렬의 복사본
#     user_inds:    훈련 데이터에서 0으로 바뀐 유저의 index
#     '''
#     test_set = matrix.copy()
#     test_set[test_set !=0] = 1 # binary하게 만들기
    
#     training_set = matrix.copy()
#     nonzero_inds = training_set.nonzero()
#     nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1]))
    
#     random.seed(0)
#     num_samples = int(np.ceil(percentage * len(nonzero_pairs)))
#     samples = random.sample (nonzero_pairs, num_samples)
    
#     user_inds = [index[0] for index in samples]
#     item_inds = [index[1] for index in samples]
    
#     training_set[user_inds, item_inds] = 0
#     training_set.eliminate_zeros()
    
#     return training_set, test_set, list(set(user_inds))


In [30]:
####### Implicit library

In [31]:
!pip install implicit

     |████████████████████████████████| 18.5 MB 424 kB/s 


In [32]:
import implicit

/usr/local/lib/python3.7/dist-packages/implicit/gpu/__init__.py:14: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  f"CUDA extension is built, but disabling GPU support because of '{e}'",


In [33]:
model = implicit.als.AlternatingLeastSquares(factors=20)

/usr/local/lib/python3.7/dist-packages/implicit/utils.py:27: UserWarning: OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading
  "OpenBLAS detected. Its highly recommend to set the environment variable "


In [34]:
user_id_mapped = 8
model.recalculate_user(user_id_mapped,purchase_sparse[user_id_mapped]).shape


이ㅏㅓ리나ㅓ

AttributeError: ignored

In [35]:
model.fit(purchase_sparse)

  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
user_id_mapped = 8

article_id, article_w = model.recommend(user_id_mapped,product_train[user_id_mapped], N=len(articles)+1, filter_already_liked_items=True)
article_w[-50:]

In [ ]:
## to make Feature user, item

In [54]:
# ratings['customer_id'].unique()

# cus_mapping

# for customer_id in :
#     user_id_mapped = cus_mapping[customer_id]
#     model.recalculate_user(user_id_mapped,purchase_sparse[user_id_mapped]).shape


def make_user_feature(customer_id):
    user_id_mapped = cus_mapping[customer_id]

    return model.recalculate_user(user_id_mapped,purchase_sparse[user_id_mapped])


def make_item_feature(article_id):
    art_id_mapped = art_mapping[article_id]
    
    return model.recalculate_item(art_id_mapped,purchase_sparse[user_id_mapped])

In [105]:
model.recalculate_item(art_id_mapped, ratings.query(f' article_id == { sdf }')['customer_id'].unique())

AttributeError: ignored

In [ ]:
ratings.query(f' article_id == { sdf }')['customer_id'].unique()

In [98]:
sdf = ratings['article_id'].values[1]

In [92]:
purchase_sparse.transpose()[31000]

<1x590906 sparse matrix of type '<class 'numpy.int64'>'
	with 143 stored elements in Compressed Sparse Column format>

In [80]:


art_id_mapped = art_mapping[ratings['article_id'].values[1]]

In [56]:
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())

0it [00:00, ?it/s]


In [106]:
customer_df = ratings['customer_id'].drop_duplicates().to_frame()
customer_df['customer_feature'] = customer_df['customer_id'].progress_apply(lambda x: make_user_feature(x))

100%|██████████| 590906/590906 [16:21<00:00, 602.17it/s]


In [59]:
customer_df.to_pickle("/content/drive/MyDrive/colab_data/kaggle_H&M/data/customer_feature.pkl")


In [60]:
pd.read_pickle("/content/drive/MyDrive/colab_data/kaggle_H&M/data/customer_feature.pkl")

,customer_id,customer_feature
26788324,33107add0f51e74f63836f005ad841955de5655ef9b755...,"(20,)"
26788326,3322e39c235a1891d20c9f1c696f02c209cf5a5b2efbe7...,"(20,)"
26788328,3325c5eefa1d842c572f8c66249e26e016c1fdca19c1da...,"(20,)"
26788330,33330851091f85fd8cf0f54407e056f180d7ed78864700...,"(20,)"
26788337,33393acc84b3f6246687b3af4259ea958797f2403ad02f...,"(20,)"
...,...,...
31788275,ff5b8a8b26bf93a66290e9bd1b73393ac6a58968a78519...,"(20,)"
31788281,ff6f55a51af284b71dcd264396b299e548f968c1769e71...,"(20,)"
31788283,ff732b98019465f078144bae647d7d9765d76e3431ef92...,"(20,)"
31788295,ffb72741f3bc3d98855703b55d34e05bc7893a5d6a99a3...,"(20,)"


In [38]:
cus_mapping[ratings['customer_id'].unique()[0]]

117687

In [ ]:
def enference(customer_id):
    try:
        target_cus = cus_mapping[customer_id]
    except:
        return 0

   
        
    predicted_labels, _ = model.recommend(target_cus, purchase_sparse[target_cus], N=12, filter_already_liked_items=False)

    return [art_inverse_mapping[x] for x in predicted_labels]

In [ ]:


import gc

valid_df = valid_ratings.sort_values(['customer_id', 't_dat'], ascending = [True, True]) 
valid_cust = valid_df.groupby('customer_id')['article_id'].apply(list).reset_index()
valid_cust['valid_true'] = valid_cust['article_id'].map(lambda x: '0'+' 0'.join(str(x)[1:-1].split(', ')))
del valid_df, valid_cust['article_id']
_ = gc.collect()

valid_cust.tail()

In [ ]:
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())

In [ ]:
sub_df = pd.read_csv('/content/drive/MyDrive/colab_data/kaggle_H&M/data/sample_submission.csv')

In [ ]:
sub_df['prediction'] = sub_df['customer_id'].progress_apply(lambda x: enference(x))

In [ ]:
sub_df['prediction'] = sub_df['prediction'].map(lambda x: '0'+' 0'.join(str(x)[1:-1].split(', ')))
sub_df

In [ ]:
sub_df.to_csv('/content/drive/MyDrive/colab_data/kaggle_H&M/data/ALS_submission.csv', index = False)

In [ ]:
valid_cust['prediction'] = valid_cust['customer_id'].progress_apply(lambda x: enference(x))

In [ ]:
valid_cust

In [ ]:
valid_cust['valid_pred'] = valid_cust['prediction'].map(lambda x: '0'+' 0'.join(str(x)[1:-1].split(', ')))
valid_cust

In [ ]:

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    # remove this case in advance
    # if not actual:
    #     return 0.0

    return score / min(len(actual), k)


def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [ ]:
mapk(
    valid_cust['valid_true'].map(lambda x: x.split()), 
    valid_cust['valid_pred'].map(lambda x: x.split()), 
    k=12
)

In [ ]:
valid_cust

In [ ]:
ratings = ratings.groupby('customer_id')['article_id'].apply(list).reset_index()
ratings['history'] = ratings['article_id'].map(lambda x: '0'+' 0'.join(str(x)[1:-1].split(', ')))
ratings

In [ ]:
df = pd.merge(valid_cust, ratings, on='customer_id')

In [ ]:
df

In [ ]:
mapk(
    df['history'].map(lambda x: x.split()), 
    df['valid_pred'].map(lambda x: x.split()), 
    k=12
)

In [ ]:
def neg_nference(customer_id):
    try:
        target_cus = cus_mapping[customer_id]
    except:
        target_cus = 1

    if target_cus:
        
        predicted_labels, _ = model.recommend(target_cus, purchase_sparse[target_cus], N=len(articles), filter_already_liked_items=True)
        np.random.choice(predicted_labels[-8000], 50, replace=False)
    return [art_inverse_mapping[x] for x in predicted_labels]

In [ ]:
valid_cust['neg_sample'] = valid_cust['customer_id'].progress_apply(lambda x: neg_nference(x))

In [ ]:
valid_cust

In [ ]:
valid_cust

In [ ]:
np.random.choice(_, 5, replace=False)

In [ ]:
_